# IMPORT

In [1]:
!pip install seaborn==0.10.1
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages (0.23.1)


In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import psutil
import re

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin

# magic word for producing visualizations in notebook
%matplotlib inline

# CUSTOM FUNCTIONS

In [2]:
from utils.custom_transformers import cleaning, naning, low_freq
from utils.helper import *

# DATA STORAGE TO S3

In [3]:
import sagemaker
session = sagemaker.session.Session
bucket = 'auto-ml-exploration'

In [13]:
s3_dataset_path = f's3://{bucket}/dataset'

sagemaker.s3.S3Uploader.upload('Udacity_AZDIAS_052018.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_CUSTOMERS_052018.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_MAILOUT_052018_TEST.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_MAILOUT_052018_TRAIN.csv', s3_dataset_path)

# LOAD DATA

## Data location

In [5]:
# data location on S3
pop_dataset_loc = f's3://{bucket}/dataset/Udacity_AZDIAS_052018.csv'
customers_dataset_loc = f's3://{bucket}/dataset/Udacity_CUSTOMERS_052018.csv'

## Partial loading

In [6]:
# partial loading
# load in the data, first only a few rows to get an idea
# I had a memory issue when loading the complete file
# 1 every 2 rows
pop_df_partial = pd.read_csv(pop_dataset_loc, sep=';', skiprows = lambda x: x%2) # skiprows because much data... Let's see if one can convert some to categorical to reduce memory usage

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
customers_df_partial = pd.read_csv(customers_dataset_loc, sep=';', skiprows = lambda x: x%2)

## Mini loading

In [ ]:
pop_df_mini = pop_df_partial.iloc[0:100,:]
customers_df_partial = customers_df_partial.iloc[0:100,:]

## Full loading

In [ ]:
pop_df = pd.read_csv(pop_dataset_loc, sep=';', dtype = cat_col)
cat_col_customers = cat_col.copy()
cat_col_customers['CUSTOMER_GROUP'] = 'category'
cat_col_customers['ONLINE_PURCHASE'] = 'category'
cat_col_customers['PRODUCT_GROUP'] = 'category'

customers_df = pd.read_csv(customers_dataset_loc, sep=';', dtype = cat_col_customers)

## Data selection

In [7]:
pop_df_selected = pop_df_partial

In [1]:
pop_df_selected = pd.read_pickle('pop_df_exploration.pkl')

NameError: name 'pd' is not defined

In [ ]:
customers_df_selected = customers_df_partial

# Clean Data

Following data cleaning is performed based on the insights gained from data exploration notebook

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import psutil
import re

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin

# magic word for producing visualizations in notebook
%matplotlib inline

from utils.custom_transformers import cleaning, naning

In [2]:
pop_df_selected = pd.read_pickle('pop_df_exploration.pkl')

In [3]:
all_columns = pop_df_selected.columns

# ! naning class DOES NOT respect scikit-learn conventions
# since it returns a pandas.DataFrame and not a numpy.array
clean_pipeline = make_pipeline(naning(attribute_filepath = 'DIAS Attributes - Values 2017.xlsx'),
                               cleaning(attribute_filepath = 'DIAS Attributes - Values 2017.xlsx'))

transformers = [
('all', clean_pipeline, all_columns),
]

combined_pipe = ColumnTransformer(transformers, remainder='drop')

In [4]:
transformed_data = combined_pipe.fit_transform(pop_df_selected)

88 replacements made
columns
AGER_TYP
ALTER_KIND1
ALTER_KIND2
ALTER_KIND3
ALTER_KIND4
D19_BANKEN_ANZ_12
D19_BANKEN_ANZ_24
D19_BANKEN_DATUM
D19_BANKEN_OFFLINE_DATUM
D19_BANKEN_ONLINE_DATUM
D19_GESAMT_ANZ_12
D19_GESAMT_OFFLINE_DATUM
D19_TELKO_ANZ_12
D19_TELKO_ANZ_24
D19_TELKO_DATUM
D19_TELKO_OFFLINE_DATUM
D19_TELKO_ONLINE_DATUM
D19_VERSAND_ANZ_12
D19_VERSAND_ANZ_24
D19_VERSAND_OFFLINE_DATUM
D19_VERSI_ANZ_12
D19_VERSI_ANZ_24
EXTSEL992
KK_KUNDENTYP
TITEL_KZ
will be dropped because theycontain a number of nan above 60.0%
columns
PRAEGENDE_JUGENDJAHRE
PRAEGENDE_JUGENDJAHRE
ANZ_STATISTISCHE_HAUSHALTE
LP_LEBENSPHASE_GROB
PRAEGENDE_JUGENDJAHRE
PRAEGENDE_JUGENDJAHRE
CJT_TYP_5
CJT_TYP_5
D19_VERSAND_ONLINE_DATUM
D19_VERSAND_ONLINE_DATUM
D19_VERSAND_ONLINE_QUOTE_12
D19_KONSUMTYP_MAX
D19_VERSAND_ONLINE_DATUM
STRUKTURTYP
FINANZ_SPARER
LP_STATUS_GROB
PRAEGENDE_JUGENDJAHRE
PRAEGENDE_JUGENDJAHRE
GEBAEUDETYP_RASTER
MOBI_REGIO
KBA05_KRSAQUOT
MOBI_REGIO
KBA05_KRSHERST1
KBA05_KRSHERST2
KBA05_KRSHERST3
KBA05

In [5]:
def get_feature_out(estimator, feature_in):
    if hasattr(estimator,'get_feature_names'):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f'vec_{f}' \
                for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in


def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []

    for name, estimator, features in ct.transformers:
        transformer = ct.named_transformers_[name]
        if name!='remainder':
            if isinstance(transformer, Pipeline):
                current_features = features
                for step in transformer:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(transformer, features)
            output_features.extend(features_out)
        elif estimator=='passthrough':
            output_features.extend(ct._feature_names_in[features])

    return output_features

In [6]:
reconstructed = pd.DataFrame(transformed_data, 
             columns=get_ct_feature_names(combined_pipe))

In [10]:
print(reconstructed.shape)

(363212, 293)


In [11]:
reconstructed.to_pickle('pop_cleaned_df.pkl')

## Removing insignificant columns - again, after cleaning

In [9]:
remove_insignificant_columns(reconstructed, thresh = 0.60)

[]

## Now Moving to scikit -> column & index will be lost since scikit works with arrays

In [3]:
reconstructed = pd.read_pickle('pop_cleaned_df.pkl')

In [4]:
# remember columns & index
pop_columns = reconstructed.columns
# customers_columns = customers_df_selected.columns

pop_index = reconstructed.index
# customers_index = customers_df_selected.index


## Identifying categorical vs numeric for further post-treatment

In [7]:
filename = 'DIAS Attributes - Values 2017.xlsx'
pop_df_numeric = identify_numeric_from_filename(filename, reconstructed) # based on Excel file
pop_df_cat = [col for col in reconstructed.columns if col not in pop_df_numeric]

In [10]:
low_freq = low_freq()
reconstructed = low_freq.fit_transform(reconstructed)

TypeError: 'low_freq' object is not callable

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder
from sklearn.impute import SimpleImputer

numeric_pipeline = make_pipeline(SimpleImputer(strategy='mean', missing_values=np.nan), Normalizer())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent', missing_values=np.nan), OneHotEncoder())

In [19]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('categorical_transformer', categorical_pipeline, pop_df_cat)]
                      )

In [20]:
X = ct.fit_transform(reconstructed)

TypeError: '<' not supported between instances of 'float' and 'str'

In [61]:
pop_df_selected.shape

(363212, 1755)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

# Complete data pipeline

# EXPERIMENTATION

In [8]:
pop_df_selected['D19_BANKEN_DATUM']

0         10
1         10
2         10
3         10
4          8
5         10
6          8
7         10
8         10
9         10
10        10
11         1
12        10
13        10
14         7
15        10
16         8
17        10
18        10
19         9
20         4
21        10
22        10
23        10
24        10
25        10
26        10
27        10
28        10
29        10
          ..
363182    10
363183    10
363184    10
363185    10
363186    10
363187     8
363188    10
363189    10
363190    10
363191     8
363192     5
363193     9
363194    10
363195    10
363196     5
363197    10
363198    10
363199    10
363200    10
363201     8
363202    10
363203    10
363204     1
363205    10
363206    10
363207    10
363208     2
363209    10
363210    10
363211    10
Name: D19_BANKEN_DATUM, Length: 363212, dtype: int64